In [1]:
%run ingest.py

wandb: Currently logged in as: sean-zhou321 (sean-zhou321-SELF). Use `wandb login --relogin` to force relogin


C:\Users\yixio\github\wandb\edu\llm-apps-course\src\ingest.py:84: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()
wandb: Adding directory to artifact (.\..\vector_store)... Done. 0.0s


In [1]:
%run eval.py

wandb: Currently logged in as: sean-zhou321 (sean-zhou321-SELF). Use `wandb login --relogin` to force relogin


sean-zhou321-SELF/llmapps/generated_examples:latest


wandb:   1 of 1 files downloaded.  
wandb:   5 of 5 files downloaded.  
wandb:   1 of 1 files downloaded.  
  0%|                                                                                           | 0/55 [00:00<?, ?it/s]C:\Users\yixio\github\wandb\edu\llm-apps-course\src\eval.py:47: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain({"question": query, "chat_history": []})
100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [02:35<00:00,  2.83s/it]
No human prompt provided. Using default human prompt from prompts
C:\Users\yixio\github\wandb\edu\llm-apps-course\src\eval.py:66: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-opena

model_accuracy,▁
model_accuracy,0.0


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import eval as this_eval
from config import default_config
from prompts import load_eval_prompt
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

import pandas as pd
eval_dataset = pd.read_csv("eval_with_answers.csv")

In [29]:
eval_prompt = load_eval_prompt()
llm = ChatOpenAI(
    model_name=default_config.eval_model,
    temperature=0,
)
chain = prompt | llm | StrOutputParser()



No human prompt provided. Using default human prompt from prompts


In [ ]:
from langchain.evaluation import load_evaluator, EvaluatorType, Criteria
evaluator = load_evaluator(evaluator=EvaluatorType.LABELED_CRITERIA, llm=llm, criteria=Criteria.CORRECTNESS)

In [ ]:
criteria = {
    "relevance": "Is the response relevant to the question?",
    "coherence": "Is the response coherent and well-structured?",
    "accuracy": "Does the response contain accurate information?",
}
eval_chain = {"query": lambda x: x["topic"], "output": chain} | load_evaluator("criteria", criteria=criteria)

result = eval_chain.invoke(
    prediction=eval_dataset["model_answer"].iloc[i], 
    reference=eval_dataset["answer"].iloc[i], 
    input=eval_dataset["question"].iloc[i]
)


In [52]:
# Create an evaluator
evaluator = load_evaluator("criteria", criteria="humor")

# Combine chain and evaluator
eval_chain = {"input": lambda x: x["topic"], "output": chain} | evaluator

i=0
examples=[{
    "query": eval_dataset["question"].iloc[i],
    "answer": eval_dataset["answer"].iloc[i],
}]
predictions=[{
    "query": eval_dataset["question"].iloc[i],
    "answer": eval_dataset["answer"].iloc[i],
    "result": eval_dataset["model_answer"].iloc[i],
}]

evaluator = LabeledCriteriaEvalChain.from_llm(llm=llm, criteria=criteria)

result = evaluator.evaluate_strings(
    prediction=eval_dataset["model_answer"].iloc[i], 
    reference=eval_dataset["answer"].iloc[i], 
    input=eval_dataset["question"].iloc[i]
)

# graded_outputs = eval_chain.evaluate(examples, predictions)
